<a href="https://colab.research.google.com/github/pgurazada/advances-in-nlp/blob/main/transfer_learning_finetune_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective

Illustrate how to fine-tune a BERT model for sentiment classification using the transformers package.

Note: This notebook should be run with a GPU. If you have access to a larger GPU, you can increase the training data size.

# Setup

In [1]:
! pip install -q datasets==2.20.0 \
                 accelerate==0.33.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.


In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from transformers import create_optimizer

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TFAutoModelForSequenceClassification

from sklearn.model_selection import train_test_split
from datasets import Dataset

2024-08-27 01:36:32.229547: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 01:36:32.229627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 01:36:32.232248: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 01:36:32.246098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 01:36:34.238176: W tensorflow/compiler/tf2

# Method

Just as with transfer learning with images, pretrained BERT models can be fine-tuned by:

- Importing a pretrained model from HuggingFace and attaching a classifier head.
- Then, we freeze the base BERT model and finetune the dense layer.
- Finally, we unfreeze the base BERT model and finetune the entire model.

The finetuned model is now ready for inference.

# Data

In [2]:
data_file = 'labeled_sentiments_data.tsv'

In [3]:
data_df = pd.read_csv(data_file, sep='\t')

In [4]:
data_df.shape

(25000, 3)

In [5]:
data_df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


# Prepare Dataset

In [6]:
train_df, test_df = train_test_split(data_df, test_size=0.2)

In [7]:
train_df.shape, test_df.shape

((20000, 3), (5000, 3))

In [8]:
sample_train_dataset = Dataset.from_pandas(train_df.sample(10000), split='train')
sample_validation_dataset = Dataset.from_pandas(test_df, split='valid')

In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
def preprocess_function(examples):
    return tokenizer(examples["review"], truncation=True)

In [11]:
tokenized_train_dataset = sample_train_dataset.map(preprocess_function, batched=True)
tokenized_validation_dataset = sample_validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [12]:
# Ensure the label column exists in the tokenized datasets
# This is an expectation by the transformers package
def add_labels(examples):
    examples['labels'] = examples['sentiment']
    return examples

In [13]:
tokenized_train_dataset = tokenized_train_dataset.map(add_labels, batched=True)
tokenized_validation_dataset = tokenized_validation_dataset.map(add_labels, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [15]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
batch_size = 24
num_epochs = 50

In [18]:
tokenized_train_dataset, tokenized_validation_dataset

(Dataset({
     features: ['id', 'sentiment', 'review', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 10000
 }),
 Dataset({
     features: ['id', 'sentiment', 'review', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 5000
 }))

In [19]:
batches_per_epoch = len(tokenized_train_dataset["sentiment"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    min_lr_ratio=0.001,
    num_warmup_steps=0,
    num_train_steps=total_train_steps
)

2024-08-27 01:38:14.519337: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-27 01:38:14.529727: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-27 01:38:14.530052: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Build Model

In [20]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [21]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Fine-Tune

Freeze the base BERT model.

In [22]:
model.layers[0].trainable = False

In [23]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 592130 (2.26 MB)
Non-trainable params: 66362880 (253.15 MB)
_________________________________________________________________


In [24]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

We use two callbacks - model checkpointing when best accuracy is observed and early stopping if validation accuracy does not improve for 4 epochs.

In [25]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "best_model",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True
)

earlystopping = tf.keras.callbacks.EarlyStopping(
    patience=4,
    monitor="val_accuracy",
    restore_best_weights=True
)

In [26]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
    tokenizer=tokenizer
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [27]:
tf_val_set = model.prepare_tf_dataset(
    tokenized_validation_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
    tokenizer=tokenizer
)

In [28]:
model.fit(
    tf_train_set,
    validation_data=tf_val_set,
    epochs=50,
    callbacks=[checkpoint, earlystopping]
)

Epoch 1/50


2024-08-27 01:40:35.339733: I external/local_xla/xla/service/service.cc:168] XLA service 0x355f9800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-27 01:40:35.339805: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P5000, Compute Capability 6.1
2024-08-27 01:40:35.347914: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-27 01:40:36.387650: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1724722836.571754     967 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 283s 425ms/step - loss: 0.6249 - accuracy: 0.7166 - val_loss: 0.5441 - val_accuracy: 0.8053
Epoch 2/50
625/625 [==============================] - 267s 427ms/step - loss: 0.4947 - accuracy: 0.8083 - val_loss: 0.4458 - val_accuracy: 0.8199
Epoch 3/50
625/625 [==============================] - 267s 427ms/step - loss: 0.4320 - accuracy: 0.8190 - val_loss: 0.4086 - val_accuracy: 0.8291
Epoch 4/50
625/625 [==============================] - 268s 429ms/step - loss: 0.3991 - accuracy: 0.8301 - val_loss: 0.3821 - val_accuracy: 0.8363
Epoch 5/50
625/625 [==============================] - 268s 428ms/step - loss: 0.3881 - accuracy: 0.8314 - val_loss: 0.3701 - val_accuracy: 0.8421
Epoch 6/50
625/625 [==============================] - 265s 424ms/step - loss: 0.3732 - accuracy: 0.8380 - val_loss: 0.3620 - val_accuracy: 0.8433
Epoch 7/50
625/625 [==============================] - 267s 427ms/step - loss: 0.3692 - accuracy: 0.8385 - val_loss: 0.3569 - val_accura

Unfreeze the base BERT model and continue training.

In [29]:
model.layers[0].trainable = True

In [30]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model.fit(
    tf_train_set,
    validation_data=tf_val_set,
    epochs=50,
    callbacks=[checkpoint, earlystopping]
)


Epoch 1/50
625/625 [==============================] - 264s 423ms/step - loss: 0.3394 - accuracy: 0.8539 - val_loss: 0.3333 - val_accuracy: 0.8566
Epoch 2/50
625/625 [==============================] - 267s 427ms/step - loss: 0.3434 - accuracy: 0.8503 - val_loss: 0.3324 - val_accuracy: 0.8586
Epoch 3/50
625/625 [==============================] - 265s 424ms/step - loss: 0.3379 - accuracy: 0.8564 - val_loss: 0.3322 - val_accuracy: 0.8582
Epoch 4/50
625/625 [==============================] - 265s 424ms/step - loss: 0.3405 - accuracy: 0.8514 - val_loss: 0.3312 - val_accuracy: 0.8586
Epoch 5/50
625/625 [==============================] - 266s 425ms/step - loss: 0.3374 - accuracy: 0.8548 - val_loss: 0.3312 - val_accuracy: 0.8590
Epoch 6/50
625/625 [==============================] - 266s 425ms/step - loss: 0.3379 - accuracy: 0.8538 - val_loss: 0.3309 - val_accuracy: 0.8594
Epoch 7/50
625/625 [==============================] - 265s 425ms/step - loss: 0.3357 - accuracy: 0.8578 - val_loss: 0.3315 -

# Inference

In [32]:
test_inputs = [
    "Awesome movie",
    "Great movie, great plot"
]

In [33]:
tokenized_inputs = tokenizer(test_inputs, return_tensors="np", padding="longest")

outputs = model(tokenized_inputs).logits

classifications = np.argmax(outputs, axis=1)

print(classifications)

[1 1]
